In deze opgave classificeren afbeeldingen van bloemen uit de iris dataset met behulp van een neuraal netwerk.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

Laad de dataset en maak een feature matrix X en een target vector y.

In [2]:
iris = sns.load_dataset('iris')

# we gebruiken .values om een Numpy array te krijgen in plaats van een Pandas DataFrame
X_iris = iris.drop('species', axis=1).values 
y_iris = iris['species'].values

In [3]:
y_iris = y_iris.reshape(-1, 1) # maak een array van array's, dit is nodig voor de volgende stap
print(y_iris[:10])

[['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']]


In [4]:
iris.head(10)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa


Een neuraal netwerk kan niet omgaan met categorische labels zoals de namen van de planten, maar heeft getallen als uitput nodig. We hebben hier drie klassen (setosa, versicolor, virginica) daarom maken we een target vector met per label drie waarden. Een 1 voor de eerste waarde correspondeert met 'setosa', een 1 voor de 2e waade met 'versicolor' en een 1 voor de derde waarde met 'virginica'.

Doe dit met behulp van het `OneHotEncoder` object uit sklearn.

In [5]:
# instantie van het OneHotEncoder-object
encoder = OneHotEncoder()

# de kolom met de labels 
labels = iris['species']

# Pas het OneHotEncoder-object toe op de kolom met de labels om de encoder te passen
encoder.fit(labels.to_numpy().reshape(-1, 1))
                                      
# Gebruik de transform-methode van het encoder-object om de kolom met de labels om te zetten naar een gecodeerde vector
encoded_labels = encoder.transform(labels.to_numpy().reshape(-1, 1)).toarray()
                                      
# Voeg de gecodeerde labels toe aan je dataset
new_iris = iris.assign(encoded_labels=encoded_labels.tolist())

# Toon de eerste vijf rijen van de dataset om te controleren of de gecodeerde labels correct zijn toegevoegd
new_iris

,sepal_length,sepal_width,petal_length,petal_width,species,encoded_labels
0,5.1,3.5,1.4,0.2,setosa,"[1.0, 0.0, 0.0]"
1,4.9,3.0,1.4,0.2,setosa,"[1.0, 0.0, 0.0]"
2,4.7,3.2,1.3,0.2,setosa,"[1.0, 0.0, 0.0]"
3,4.6,3.1,1.5,0.2,setosa,"[1.0, 0.0, 0.0]"
4,5.0,3.6,1.4,0.2,setosa,"[1.0, 0.0, 0.0]"
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,"[0.0, 0.0, 1.0]"
146,6.3,2.5,5.0,1.9,virginica,"[0.0, 0.0, 1.0]"
147,6.5,3.0,5.2,2.0,virginica,"[0.0, 0.0, 1.0]"
148,6.2,3.4,5.4,2.3,virginica,"[0.0, 0.0, 1.0]"


We hebben de data nu in een geschikt formaat. Splits de data in training en test data.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_iris, encoded_labels, test_size = 0.2, random_state = 6)


Maak een eenvoudig neuraal netwerk met 3 lagen, waarin elke neuron in een laag met elk neuron in de vorige laag verbonden is ('Dense'). 
Voeg voor de eeste laag een parameter `input_shape` toe die aangeeft hoeveel features een element in 'X' heeft.
Het aantal neuronen in de laaste laag is gelijk aan het aantal waarden in een label in `y`. 

In [7]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

# input laag
model.add(Dense(16, input_shape=(4, ), activation = 'relu'))

# verborgen laag toe
model.add(Dense(16, activation='relu'))

# extra verborgen laag toe met 32 knooppunten
model.add(Dense(32, activation='relu'))

# output laag toe
model.add(Dense(3, activation='softmax'))

Compileer en bekijk het netwerk door onderstaande code te runnen.

In [8]:
from keras.optimizers import Adam

model.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                80        
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 32)                544       
                                                                 
 dense_3 (Dense)             (None, 3)                 99        
                                                                 
Total params: 995
Trainable params: 995
Non-trainable params: 0
_________________________________________________________________


C:\Users\Aghoo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Nu kunnen we ons model trainen met behulp van de `fit` methode.

In [9]:
model.fit(X_train, y_train, epochs = 25, batch_size = 5)

Epoch 1/25
24/24 [==============================] - 1s 3ms/step - loss: 0.7943 - accuracy: 0.6583
Epoch 2/25
24/24 [==============================] - 0s 3ms/step - loss: 0.5177 - accuracy: 0.7333
Epoch 3/25
24/24 [==============================] - 0s 3ms/step - loss: 0.4187 - accuracy: 0.7917
Epoch 4/25
24/24 [==============================] - 0s 3ms/step - loss: 0.2518 - accuracy: 0.9417
Epoch 5/25
24/24 [==============================] - 0s 3ms/step - loss: 0.1608 - accuracy: 0.9583
Epoch 6/25
24/24 [==============================] - 0s 3ms/step - loss: 0.2232 - accuracy: 0.9167
Epoch 7/25
24/24 [==============================] - 0s 3ms/step - loss: 0.1506 - accuracy: 0.9250
Epoch 8/25
24/24 [==============================] - 0s 3ms/step - loss: 0.1022 - accuracy: 0.9500
Epoch 9/25
24/24 [==============================] - 0s 3ms/step - loss: 0.2803 - accuracy: 0.9000
Epoch 10/25
24/24 [==============================] - 0s 3ms/step - loss: 0.1682 - accuracy: 0.9417
Epoch 11/25
24/24 [

Met `model_evaluate` kunnen we bepalen hoe goed het model werkt op de test data.

In [10]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(test_acc)

1/1 [==============================] - 0s 369ms/step - loss: 0.0715 - accuracy: 0.9667
0.9666666388511658


Experimenteer met bovenstaand model. Wat gebeurt er als je meer lagen toevoegt of een laag weghaalt of het aantal neuronen in een laag verandert? 

Probeer ook eens andere activatiefuncties dan `relu`, zoals `tanh` of `sigmoid`.

Welke invloed heeft het aantal epochs?

- In eerste instantie heb ik de activatiefuncties op relu gezet en de grootte van de knooppunten aangepast. Echter, er was geen verandering in het aantal epochs of in de model.evaluate score.

- Vervolgens heb ik ervoor gekozen om alle activatiefuncties op tanh te zetten. Hierbij merkte ik direct een verschil in de model.evaluate score, deze was namelijk lager. Echter, bij het gebruik van meer knooppunten werd de model.evaluate score gelijk aan die van relu.

Op basis van deze resultaten heb ik besloten om de relu activatiefuncties te gebruiken, omdat ik hierbij hetzelfde test_acc score bereik met minder knooppunten.

We kunnen dezelfde aanpak gebruiken voor andere data, bijvoorbeeld voor het herkennen van cijfers. We gebruiken de `MNist` dataset die bestaat uit plaatjes van cijfers. Ieder plaatje bestaat uit 28 x 28 pixels.

In [26]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

import warnings
warnings.filterwarnings('ignore')

In [27]:
train_images.shape

(60000, 28, 28)

In [28]:
len(train_labels)

60000

In [32]:
print(train_labels)

[5 0 4 ... 5 6 8]


Een eenvoudig neuraal netwerk heeft een array van inputwaarden nodig i.p.v. een 2D afbeelding.
We maken van de inputs (waarden tussen 0 en 255) getallen tussen 0 en 1.

In [15]:
train_images = train_images.reshape((60000, 28 * 28)) # lijst van waarden i.p.v. 2D afbeelding
train_images = train_images.astype('float32') / 255 # getallen tussen 0 en 1

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

Voor elk label (een cijfer) maken we een array met 10 waarden: 9 nullen en een één, waarbij de positie van de één aangeeft om welk cijfer het gaat (One hot encoding). Dit keer gebruiken we hiervoor de `keras` functie `to_categorical`.

In [16]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print(train_labels)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


Maak nu een neuraal netwerk met twee lagen, een eerste laag met 512 neuronen en een output laag met 10 neuronen. Geef bij de eerste laag aan hoeveel inputs er zijn en wat de activatiefunctie is. De output laag heeft een `softmax` activatiefunctie.

In [17]:
model = Sequential()

# input laag
model.add(Dense(512, input_shape=(784, ), activation = 'relu'))

# output laag toe
model.add(Dense(10, activation='softmax'))

compileer het netwerk.

In [18]:
model.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 512)               401920    
                                                                 
 dense_5 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


train het netwerk 5 epochs met een batch_size van 128.

In [19]:
model.fit(train_images, train_labels, epochs = 5, batch_size = 128)

Epoch 1/5
469/469 [==============================] - 5s 10ms/step - loss: 0.2072 - accuracy: 0.9365
Epoch 2/5
469/469 [==============================] - 5s 10ms/step - loss: 0.1109 - accuracy: 0.9681
Epoch 3/5
469/469 [==============================] - 5s 10ms/step - loss: 0.0846 - accuracy: 0.9748
Epoch 4/5
469/469 [==============================] - 5s 10ms/step - loss: 0.0792 - accuracy: 0.9776
Epoch 5/5
469/469 [==============================] - 5s 10ms/step - loss: 0.0732 - accuracy: 0.9798


Evalueer het netwerk op de test set.

In [20]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

313/313 [==============================] - 2s 4ms/step - loss: 0.1534 - accuracy: 0.9661
0.9660999774932861
